In [81]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_absolute_percentage_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [82]:
from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [83]:
bist100 = pd.read_csv("RELIANCE.csv")
bist100.head()
# bist100=bist100.drop(['Symbol', 'Series','Prev Close','Last','VWAP','Turnover','Trades','Deliverable Volume','%Deliverble'], axis=1)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2010-01-04,ASIANPAINT,EQ,1797.20,1791.00,1799.90,1780.00,1787.6,1788.65,1790.13,8035,1.438371e+12,NaN,4852.0,0.6039
1,2010-01-05,ASIANPAINT,EQ,1788.65,1798.00,1804.95,1785.00,1796.0,1794.30,1794.78,84910,1.523944e+13,NaN,80968.0,0.9536
2,2010-01-06,ASIANPAINT,EQ,1794.30,1814.75,1814.75,1781.00,1781.4,1788.90,1793.06,7345,1.317006e+12,NaN,4568.0,0.6219
3,2010-01-07,ASIANPAINT,EQ,1788.90,1798.75,1798.75,1769.00,1773.9,1770.65,1776.99,24032,4.270457e+12,NaN,20397.0,0.8487
4,2010-01-08,ASIANPAINT,EQ,1770.65,1774.50,1834.00,1770.65,1804.0,1799.90,1804.96,71131,1.283883e+13,NaN,60998.0,0.8575


In [84]:
bist100.rename(columns={"Date":"date","Open":"open","High":"high","Low":"low","Close":"close"}, inplace= True)
bist100.head()

,date,Symbol,Series,Prev Close,open,high,low,Last,close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2010-01-04,ASIANPAINT,EQ,1797.20,1791.00,1799.90,1780.00,1787.6,1788.65,1790.13,8035,1.438371e+12,NaN,4852.0,0.6039
1,2010-01-05,ASIANPAINT,EQ,1788.65,1798.00,1804.95,1785.00,1796.0,1794.30,1794.78,84910,1.523944e+13,NaN,80968.0,0.9536
2,2010-01-06,ASIANPAINT,EQ,1794.30,1814.75,1814.75,1781.00,1781.4,1788.90,1793.06,7345,1.317006e+12,NaN,4568.0,0.6219
3,2010-01-07,ASIANPAINT,EQ,1788.90,1798.75,1798.75,1769.00,1773.9,1770.65,1776.99,24032,4.270457e+12,NaN,20397.0,0.8487
4,2010-01-08,ASIANPAINT,EQ,1770.65,1774.50,1834.00,1770.65,1804.0,1799.90,1804.96,71131,1.283883e+13,NaN,60998.0,0.8575


In [85]:
bist100.dropna(inplace=True)
bist100.isnull().sum()

date                  0
Symbol                0
Series                0
Prev Close            0
open                  0
high                  0
low                   0
Last                  0
close                 0
VWAP                  0
Volume                0
Turnover              0
Trades                0
Deliverable Volume    0
%Deliverble           0
dtype: int64

In [86]:
# convert date field from string to Date format and make it index
bist100['date'] = pd.to_datetime(bist100.date)
bist100.head()

,date,Symbol,Series,Prev Close,open,high,low,Last,close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
354,2011-06-01,ASIANPAINT,EQ,3133.40,3145.10,3194.00,3061.50,3170.00,3169.35,3156.21,192349,6.070944e+13,11665.0,89136.0,0.4634
355,2011-06-02,ASIANPAINT,EQ,3169.35,3159.00,3171.80,3087.00,3110.05,3105.05,3133.25,211697,6.633002e+13,6690.0,169251.0,0.7995
356,2011-06-03,ASIANPAINT,EQ,3105.05,3105.05,3190.00,3099.95,3181.00,3167.80,3153.77,83355,2.628821e+13,5398.0,45782.0,0.5492
357,2011-06-06,ASIANPAINT,EQ,3167.80,3160.20,3188.85,3141.25,3169.50,3170.05,3163.77,46291,1.464539e+13,4838.0,20367.0,0.4400
358,2011-06-07,ASIANPAINT,EQ,3170.05,3170.10,3229.80,3155.00,3196.00,3206.15,3202.18,82014,2.626232e+13,8868.0,27403.0,0.3341


In [87]:
bist100.sort_values(by='date', inplace=True)
bist100.head()
bist100.shape


(2456, 15)

In [88]:
closedf = bist100[['date','close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (2456, 2)


In [89]:
close_stock = closedf.copy()
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(2456, 1)


In [90]:
# from sklearn.ensemble import RandomForestRegressor

# regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
# regressor.fit(X_train, y_train)

In [91]:
training_size=int(len(closedf)*0.65)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (1596, 1)
test_data:  (860, 1)


In [92]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [93]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (1580, 15)
y_train:  (1580,)
X_test:  (844, 15)
y_test (844,)


In [94]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [95]:
train_predict=regressor.predict(X_train)
test_predict=regressor.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

Train data prediction: (1580, 1)
Test data prediction: (844, 1)


In [96]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

In [97]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))
print("Test data MAPE: ",mean_absolute_percentage_error(original_ytest,test_predict)*100)

Train data RMSE:  52.24386680111903
Train data MSE:  2729.4216183330677
Test data MAE:  10.778562974683592
-------------------------------------------------------------------------------------
Test data RMSE:  336.0651232791514
Test data MSE:  112939.76708463122
Test data MAE:  260.63954620853127
Test data MAPE:  14.999240069247424


In [98]:
print("Train data explained variance regression score:", explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", explained_variance_score(original_ytest, test_predict))



Train data explained variance regression score: 0.9987041501122393
Test data explained variance regression score: 0.47082255057660805


In [99]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.998703974653116
Test data R2 score: 0.370163212055153


In [100]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0012772226301325366
Test data MGD:  0.04845891996180806
----------------------------------------------------------------------
Train data MPD:  1.6767512706063044
Test data MPD:  72.0674791663847


In [101]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['date'],
                       'original_close': close_stock['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


Train predicted data:  (2456, 1)
Test predicted data:  (2456, 1)
